In [1]:
# Importing the required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error,  r2_score
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import kerastuner as kt
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Loading the dataset
# Train Data
temp_df = pd.read_csv("../../data/cleaned/train.csv", nrows=0)  # Read only the header
total_columns = len(temp_df.columns)
columns_to_use = temp_df.columns[1:total_columns]
train_data = pd.read_csv("../../data/cleaned/train.csv",  usecols=columns_to_use)
print(train_data.head(5))

# Test Data
test_data = pd.read_csv("../../data/cleaned/test.csv", usecols=columns_to_use)
print(test_data.head())

# Dropping the columns that are not relevant to our analysis
train_data = train_data.drop(columns=['building_name', 'site_name'])
test_data = test_data.drop(columns=['building_name', 'site_name'])

train_data = train_data[(train_data['meter'] == 'electricity') | (train_data['meter'] == 'chilledwater') | (train_data['meter'] == 'steam')
| (train_data['meter'] == 'hotwater') | (train_data['meter'] == 'gas')]

test_data = test_data[(test_data['meter'] == 'electricity') | (test_data['meter'] == 'chilledwater') | (test_data['meter'] == 'steam')
| (test_data['meter'] == 'hotwater') | (test_data['meter'] == 'gas')]

# Building index on building_id for further assessment
train_data.set_index('building_id', inplace=True)
test_data.set_index('building_id', inplace=True)

            building_name        meter        date  meter_reading site_name  \
0  Bear_education_Alfredo  electricity  2016-01-01         2.9050      Bear   
1  Bear_education_Alfredo  electricity  2016-01-02         2.7700      Bear   
2  Bear_education_Alfredo  electricity  2016-01-03         2.6725      Bear   
3  Bear_education_Alfredo  electricity  2016-01-04         4.5650      Bear   
4  Bear_education_Alfredo  electricity  2016-01-05         4.7825      Bear   

  sub_primaryspaceusage    sqm    sqft    timezone  airTemperature  \
0             Education  609.8  6564.0  US/Pacific        5.246861   
1             Education  609.8  6564.0  US/Pacific        5.993973   
2             Education  609.8  6564.0  US/Pacific        5.660314   
3             Education  609.8  6564.0  US/Pacific        5.048507   
4             Education  609.8  6564.0  US/Pacific        4.745567   

   cloudCoverage  dewTemperature  precipDepth1HR  precipDepth6HR  \
0       1.927009        0.254484    

In [3]:
# Filtering for electrity meter_reading
train_data = train_data[train_data['meter'] == 'electricity']
test_data = test_data[test_data['meter'] == 'electricity']

train_data = train_data.drop(columns=['meter'])
test_data = test_data.drop(columns=['meter'])

In [4]:
# Inspecting the data frames
print(train_data.sample(2))
print('-------------------------------------------------------------')
print(test_data.sample(2))

                   date  meter_reading sub_primaryspaceusage      sqm  \
building_id                                                             
489          2016-05-16       1653.660           K-12 School   8092.5   
222          2016-07-10      15689.953              Research  18064.2   

                 sqft    timezone  airTemperature  cloudCoverage  \
building_id                                                        
489           87107.0  US/Eastern       14.957395       1.774587   
222          194441.0  US/Eastern       22.484152       2.022821   

             dewTemperature  precipDepth1HR  precipDepth6HR  seaLvlPressure  \
building_id                                                                   
489                6.224724        0.585795       12.354488     1016.815487   
222               14.543750        0.803643       17.650906     1011.892552   

             windDirection  windSpeed  season  site_id  
building_id                                             
489

In [5]:
# Separating into X and Y dataframes
X_train = train_data.drop(columns=['meter_reading'])  # Exclude target variable
y_train = train_data['meter_reading']

X_test = test_data.drop(columns=['meter_reading'])  # Exclude target variable
y_test = test_data['meter_reading']

In [6]:
# Convert 'site_id' from numeric to categorical
X_train['site_id'] = X_train['site_id'].astype('category')
X_test['site_id'] = X_test['site_id'].astype('category')

In [7]:
print(X_train.dtypes)
print(X_train.columns)

date                       object
sub_primaryspaceusage      object
sqm                       float64
sqft                      float64
timezone                   object
airTemperature            float64
cloudCoverage             float64
dewTemperature            float64
precipDepth1HR            float64
precipDepth6HR            float64
seaLvlPressure            float64
windDirection             float64
windSpeed                 float64
season                     object
site_id                  category
dtype: object
Index(['date', 'sub_primaryspaceusage', 'sqm', 'sqft', 'timezone',
       'airTemperature', 'cloudCoverage', 'dewTemperature', 'precipDepth1HR',
       'precipDepth6HR', 'seaLvlPressure', 'windDirection', 'windSpeed',
       'season', 'site_id'],
      dtype='object')


In [8]:
# Define features and types based on your dataset
numerical_features = ['sqm', 'sqft', 'airTemperature', 'cloudCoverage', 'dewTemperature',
                      'precipDepth1HR', 'precipDepth6HR', 'seaLvlPressure', 'windDirection', 'windSpeed']
categorical_features = ['timezone', 'season', 'sub_primaryspaceusage', 'site_id']

In [9]:
# Create a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [10]:
# Create a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Fit the preprocessor on the training data and transform both training and test data
preprocessor.fit(X_train)
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [13]:
# Convert the processed data back to dense DataFrames
X_train_processed_df = pd.DataFrame(X_train_processed.toarray(), columns=preprocessor.get_feature_names_out())
X_test_processed_df = pd.DataFrame(X_test_processed.toarray(), columns=preprocessor.get_feature_names_out())

In [14]:
# Checking the columns
X_train_processed_df.columns

Index(['num__sqm', 'num__sqft', 'num__airTemperature', 'num__cloudCoverage',
       'num__dewTemperature', 'num__precipDepth1HR', 'num__precipDepth6HR',
       'num__seaLvlPressure', 'num__windDirection', 'num__windSpeed',
       'cat__timezone_Europe/Dublin', 'cat__timezone_Europe/London',
       'cat__timezone_US/Central', 'cat__timezone_US/Eastern',
       'cat__timezone_US/Mountain', 'cat__timezone_US/Pacific',
       'cat__season_Fall', 'cat__season_Spring', 'cat__season_Summer',
       'cat__season_Winter', 'cat__sub_primaryspaceusage_Academic',
       'cat__sub_primaryspaceusage_Auditorium',
       'cat__sub_primaryspaceusage_Classroom',
       'cat__sub_primaryspaceusage_College Classroom',
       'cat__sub_primaryspaceusage_College Laboratory',
       'cat__sub_primaryspaceusage_Education',
       'cat__sub_primaryspaceusage_K-12 School',
       'cat__sub_primaryspaceusage_Other - Education',
       'cat__sub_primaryspaceusage_Primary/Secondary Classroom',
       'cat__sub_pri

In [15]:
y_train_scaled = np.log1p(y_train.values.reshape(-1, 1))
y_test_scaled = np.log1p(y_test.values.reshape(-1, 1))

### Neural Networks

In [16]:
# Define the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train_processed_df.shape[1], activation='relu')) # Adjust input_dim based on your features
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))  # Output layer, adjust units and activation based on your output

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

In [17]:
# Fit the model to the training data
model.fit(X_train_processed, y_train_scaled, epochs=10, batch_size=10, verbose=1)

# Predict the values for X_train
y_pred = model.predict(X_train_processed_df)

Epoch 1/10


2024-03-06 20:32:15.757284: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 218164 of 221064


  344/22107 [..............................] - ETA: 9s - loss: 10.9039    

2024-03-06 20:32:16.940209: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


22107/22107 [==============================] - 19s 339us/step - loss: 3.8568
Epoch 2/10


...

22107/22107 [==============================] - 18s 341us/step - loss: 3.2809
Epoch 3/10
22107/22107 [==============================] - 16s 347us/step - loss: 3.0865
Epoch 4/10
22107/22107 [==============================] - 10s 341us/step - loss: 2.9674
Epoch 5/10
22107/22107 [==============================] - 10s 339us/step - loss: 2.8750
Epoch 6/10
22107/22107 [==============================] - 9s 339us/step - loss: 2.7754
Epoch 7/10
22107/22107 [==============================] - 9s 341us/step - loss: 2.6983
Epoch 8/10


2024-03-06 20:33:45.914662: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 188888 of 221064


  205/22107 [..............................] - ETA: 16s - loss: 2.5372 

2024-03-06 20:33:46.706228: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


22107/22107 [==============================] - 18s 347us/step - loss: 2.6270
Epoch 9/10
22107/22107 [==============================] - 13s 527us/step - loss: 2.5682
Epoch 10/10
6909/6909 [==============================] - 2s 215us/step


In [18]:
#mae = mean_absolute_error(y_train_scaled, y_pred)
mse = mean_squared_error(y_train_scaled, y_pred)
r2 = r2_score(y_train_scaled, y_pred)

#print('Mean Absolute Error:', mae)
print('Mean Squared Error:', mse)
print('R-squared Score:', r2)


Mean Squared Error: 2.438231536056621
R-squared Score: 0.6237545490626539


In [19]:
# Predict the values for X_test
y_pred = model.predict(X_test_processed_df)
y_pred = np.nan_to_num(y_pred, nan=0)
mse = mean_squared_error(y_test_scaled, y_pred)
r2 = r2_score(y_test_scaled, y_pred)
print('Mean Squared Error:', mse)
print('R-squared Score:', r2)

6890/6890 [==============================] - 2s 217us/step
Mean Squared Error: 7.407310663508326
R-squared Score: 0.15225681169141358


In [26]:


def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(
        units=hp.Int('input_units', min_value=32, max_value=256, step=64),  # Reduced max value and increased step
        activation='relu',
        input_shape=(X_train_processed.shape[1],)
    ))

    # Reduced the maximum number of layers
    for i in range(hp.Int('n_layers', 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f'layer_{i}_units', min_value=32, max_value=256, step=64),  # Simplify layers
            activation='relu'
        ))

    model.add(layers.Dense(1)) 

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3, 1e-4])),  # Simplified choices
        loss='mse',  
        metrics=['mae', 'mse'] 
    )

    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_mse', 
    max_trials=10,  
    executions_per_trial=2,  
    directory='my_dir',
    project_name='quick_tune'
)

tuner.search(
    X_train_processed_df, y_train_scaled,
    epochs=5,  # Reduced number of epochs for tuning
    validation_split=0.1,  # Adjust based on your dataset
    batch_size=256  # Increased batch size for faster processing
)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
history = model.fit(
    X_train_processed_df, y_train_scaled,
    epochs=10,  # You can increase this for the final model training
    validation_split=0.1,  # Keep consistent with tuning phase or adjust as necessary
    batch_size=256  # Set a fixed batch size
)

Trial 10 Complete [00h 00m 08s]
val_mse: 4.106049060821533

Best val_mse So Far: 4.106049060821533
Total elapsed time: 00h 01m 17s
Epoch 1/10
778/778 [==============================] - 1s 876us/step - loss: 12.1278 - mae: 2.5511 - mse: 12.1278 - val_loss: 3.7031 - val_mae: 1.4944 - val_mse: 3.7031
Epoch 2/10
778/778 [==============================] - 1s 770us/step - loss: 5.0389 - mae: 1.4893 - mse: 5.0389 - val_loss: 3.5173 - val_mae: 1.4478 - val_mse: 3.5173
Epoch 3/10
778/778 [==============================] - 1s 756us/step - loss: 4.7162 - mae: 1.4159 - mse: 4.7162 - val_loss: 3.2450 - val_mae: 1.3202 - val_mse: 3.2450
Epoch 4/10
778/778 [==============================] - 1s 753us/step - loss: 4.5200 - mae: 1.3741 - mse: 4.5200 - val_loss: 3.1111 - val_mae: 1.2770 - val_mse: 3.1111
Epoch 5/10
778/778 [==============================] - 1s 783us/step - loss: 4.3767 - mae: 1.3409 - mse: 4.3767 - val_loss: 3.2783 - val_mae: 1.3504 - val_mse: 3.2783
Epoch 6/10
778/778 [=================

In [27]:
# Check and replace NaN values in the test features 
if X_test_processed_df.isna().any().any(): 
    X_test_processed_df = X_test_processed_df.fillna(0)  # Replace NaN with 0 

# Check and replace NaN values in the test labels 
if np.isnan(y_test_scaled).any():  
    y_test_scaled = np.nan_to_num(y_test_scaled)  # Replace NaN with 0 (default)

In [28]:
test_loss, test_mae, test_mse = model.evaluate(X_test_processed_df, y_test_scaled, batch_size=256)
print(f'Test Loss: {test_loss}, Test MAE: {test_mae}, Test MSE: {test_mse}')

862/862 [==============================] - 0s 354us/step - loss: 6.9401 - mae: 1.6631 - mse: 6.9401
Test Loss: 6.94014835357666, Test MAE: 1.6630945205688477, Test MSE: 6.94014835357666


In [29]:
y_pred = model.predict(X_test_processed_df, batch_size=256)

# Calculate R-squared score
r_squared = r2_score(y_test_scaled, y_pred)
print(f'R-squared Score: {r_squared}')

862/862 [==============================] - 0s 347us/step
R-squared Score: 0.20572227806479382
